In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

In [2]:
tf.set_random_seed(777) 
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
char_set = list(set (sentence)) #index -> char
char_dic = {c:i for i , c in enumerate(char_set)}

In [3]:
dic_size = len(char_set) #RNN input size
num_classes = len(char_set)
batch_size = 1
sequence_length = 10
learning_rate = 0.1
rnn_hidden_size = len(char_set)
data_dim = len(char_set)

In [4]:
dataX = []
dataY = []

for i in range(0,len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '- ->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)    

0 if you wan - -> f you want
1 f you want - ->  you want 
2  you want  - -> you want t
3 you want t - -> ou want to
4 ou want to - -> u want to 
5 u want to  - ->  want to b
6  want to b - -> want to bu
7 want to bu - -> ant to bui
8 ant to bui - -> nt to buil
9 nt to buil - -> t to build
10 t to build - ->  to build 
11  to build  - -> to build a
12 to build a - -> o build a 
13 o build a  - ->  build a s
14  build a s - -> build a sh
15 build a sh - -> uild a shi
16 uild a shi - -> ild a ship
17 ild a ship - -> ld a ship,
18 ld a ship, - -> d a ship, 
19 d a ship,  - ->  a ship, d
20  a ship, d - -> a ship, do
21 a ship, do - ->  ship, don
22  ship, don - -> ship, don'
23 ship, don' - -> hip, don't
24 hip, don't - -> ip, don't 
25 ip, don't  - -> p, don't d
26 p, don't d - -> , don't dr
27 , don't dr - ->  don't dru
28  don't dru - -> don't drum
29 don't drum - -> on't drum 
30 on't drum  - -> n't drum u
31 n't drum u - -> 't drum up
32 't drum up - -> t drum up 
33 t drum up  - ->  

In [5]:
batch_size = len(dataX)

In [6]:
X = tf.placeholder(tf.int32,[None, sequence_length]) #X one-hot
Y = tf.placeholder(tf.int32,[None, sequence_length]) #Y Label

In [7]:
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


In [8]:
def lstm_cell():
    cell = rnn.BasicLSTMCell(rnn_hidden_size, state_is_tuple = True)
    return cell

In [9]:
multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple = True)
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

In [10]:
X_for_fc  = tf.reshape(outputs, [-1, rnn_hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

In [11]:
softmax_w = tf.get_variable("softmax_w", [rnn_hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
softmax_outputs = tf.matmul(outputs, softmax_w) + softmax_b

In [12]:
#reshape out for squence loss
outputs = tf.reshape(softmax_outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, weights= weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(mean_loss)

prediction = tf.argmax(outputs, axis=2)

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2275):
        _, l, results = sess.run(
            [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
        for j , result in enumerate(results):
            index = np.argmax(result, axis = 1)
            #print(i , j , ''.join([char_set[t] for t in index]), l)
            
    results = sess.run(outputs, feed_dict = {X : dataX})
    for j, result in enumerate(results):
        index = np.argmax(result, axis = 1)
        if j is 0 : # print all for the first result to make a sentence
            print(''.join([char_set[t] for t in index]), end = '')
        else:
            print(char_set[index[-1]], end = '')
        

l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

In [14]:
'''
0 154 kkkkkkkkkk 3.24344
0 155 kkkk,,kkkk 3.24344
0 156 k,,,,kkkkk 3.24344
0 157 kkkkkkkkk, 3.24344
0 158 kkkkkkkk,, 3.24344
0 159 kkkkkkk,,k 3.24344
0 160 kkklll,,kk 3.24344
0 161 kkkkk,kkkk 3.24344
0 162 kkkkkkkkkk 3.24344
0 163 kkk,kkkkkk 3.24344
0 164 kk,,kkkkkk 3.24344
0 165 k,kkkkkkkk 3.24344
0 166 kkkkkkkkkk 3.24344
0 167 kkkkkkkkkk 3.24344
0 168 kkkkkkkk,, 3.24344
0 169 kkkkkkk,,, 3.24344

.....
2274 154 ecs immens 0.228647
2274 155 rs immensi 0.228647
2274 156 s immensit 0.228647
2274 157 sammensity 0.228647
2274 158 tmmensity  0.228647
2274 159 gmensity o 0.228647
2274 160  ensity of 0.228647
2274 161  nsity of  0.228647
2274 162 rsity of t 0.228647
2274 163 dity of th 0.228647
2274 164 sgy of the 0.228647
2274 165 gy of the  0.228647
2274 166   of the s 0.228647
2274 167 oof the se 0.228647
2274 168 tf the sea 0.228647
2274 169   the sea. 0.228647

'''

'\n0 154 kkkkkkkkkk 3.24344\n0 155 kkkk,,kkkk 3.24344\n0 156 k,,,,kkkkk 3.24344\n0 157 kkkkkkkkk, 3.24344\n0 158 kkkkkkkk,, 3.24344\n0 159 kkkkkkk,,k 3.24344\n0 160 kkklll,,kk 3.24344\n0 161 kkkkk,kkkk 3.24344\n0 162 kkkkkkkkkk 3.24344\n0 163 kkk,kkkkkk 3.24344\n0 164 kk,,kkkkkk 3.24344\n0 165 k,kkkkkkkk 3.24344\n0 166 kkkkkkkkkk 3.24344\n0 167 kkkkkkkkkk 3.24344\n0 168 kkkkkkkk,, 3.24344\n0 169 kkkkkkk,,, 3.24344\n\n.....\n2274 154 ecs immens 0.228647\n2274 155 rs immensi 0.228647\n2274 156 s immensit 0.228647\n2274 157 sammensity 0.228647\n2274 158 tmmensity  0.228647\n2274 159 gmensity o 0.228647\n2274 160  ensity of 0.228647\n2274 161  nsity of  0.228647\n2274 162 rsity of t 0.228647\n2274 163 dity of th 0.228647\n2274 164 sgy of the 0.228647\n2274 165 gy of the  0.228647\n2274 166   of the s 0.228647\n2274 167 oof the se 0.228647\n2274 168 tf the sea 0.228647\n2274 169   the sea. 0.228647\n\n'